In [1]:
import gdal2tiles
import glob
import numpy as np
import numpy.ma as ma
import os
from osgeo import gdal, gdalconst, osr
import rasterio
import rasterio.merge
from rasterio.windows import Window

%run _aws.ipynb
%run _constants.ipynb

In [1]:
from osgeo import gdal


def create_blank_tif(bbox_poly_ll, dst_dir):

    bounds = bbox_poly_ll.bounds
    xmin, xmax = bounds[0], bounds[2] 
    ymin, ymax = bounds[1], bounds[3]

    driver = gdal.GetDriverByName('GTiff')
    spatref = osr.SpatialReference()
    spatref.ImportFromEPSG(4326)
    wkt = spatref.ExportToWkt()

    blank_path = f'{dst_dir}/blank.tif'
    nbands = 1
    
    # FIXME: how does this hold up at higher latitudes?
    res = 10 / (111.32 * 1000)
    xres = res  # resx = 10 / (111.32 * 1000) * cos((ymax-ymin)/2)
    yres = -res
    
    transform = [xmin, xres, 0, ymax, 0, yres]

    dtype = gdal.GDT_UInt16

    xsize = int(np.rint(np.abs((xmax - xmin)) / xres))
    ysize = int(np.rint(np.abs((ymax - ymin) / yres)))

    ds = driver.Create(blank_path, xsize, ysize, nbands, dtype, options=['COMPRESS=LZW', 'TILED=YES'])
    ds.SetProjection(wkt)
    ds.SetGeoTransform(transform)
    ds.GetRasterBand(1).Fill(NODATA_UINT16)
    ds.GetRasterBand(1).SetNoDataValue(NODATA_UINT16)
    ds.FlushCache()
    ds = None
    
    return blank_path


def create_blank_tif_ea(bbox_poly_ea, dst_dir):

    bounds = bbox_poly_ea.bounds
    xmin, xmax = bounds[0], bounds[2] 
    ymin, ymax = bounds[1], bounds[3]

    driver = gdal.GetDriverByName('GTiff')
    spatref = osr.SpatialReference()
    spatref.ImportFromEPSG(3857)
    wkt = spatref.ExportToWkt()

    blank_path = f'{dst_dir}/blank.tif'
    nbands = 1
    xres = 10
    yres = -10

    dtype = gdal.GDT_UInt16

    xsize = int(np.rint(np.abs((xmax - xmin)) / xres))
    ysize = int(np.rint(np.abs((ymax - ymin) / yres)))

    ds = driver.Create(blank_path, xsize, ysize, nbands, dtype, options=['COMPRESS=LZW', 'TILED=YES'])
    ds.SetProjection(wkt)
    ds.SetGeoTransform([xmin, xres, 0, ymax, 0, yres])
    ds.GetRasterBand(1).Fill(NODATA_UINT16)
    ds.GetRasterBand(1).SetNoDataValue(NODATA_UINT16)
    ds.FlushCache()
    ds = None
    
    gdal.Warp(blank_path, blank_path, dstSRS="EPSG:4326")

    return blank_path

In [2]:
import rioxarray


def create_composite_dask(band, stack_path, dst_dir, method="median", overwrite=True):
    
    composite_path = f'{dst_dir}/{band}_composite.tif' 
    if os.path.exists(composite_path) and not overwrite:
        return composite_path
    
    if not os.path.exists(stack_path):
        raise ValueError(f'{stack_path} does not exist')
    
    with rasterio.open(stack_path) as stack_src:
        band_count = stack_src.count
        meta = stack_src.meta.copy()
        meta.update(count=1)
        
    stack = rioxarray.open_rasterio(stack_path, chunks=(band_count, 1000, 1000), mask_and_scale=True)
    
    centre = stack.median(axis=0, skipna=True)
    centre = np.rint(centre).astype(np.uint16)
    centre = centre.compute()
        
    with rasterio.open(composite_path, "w", **meta) as composite_dst:
        composite_dst.write(centre.data, indexes=1)

    return composite_path


def create_composite(band, stack_path, dst_dir, method="median", overwrite=False):
    
    if not os.path.exists(stack_path):
        raise ValueError(f'{stack_path} does not exist')

    composite_path = f'{dst_dir}/{band}_composite.tif'
    if os.path.exists(composite_path) and not overwrite:
        return composite_path        

    with rasterio.open(stack_path) as stack_src:
    
        width, height = stack_src.width, stack_src.height
        
        meta = stack_src.meta.copy()
        meta.update(count=1)
        
        with rasterio.open(composite_path, "w", **meta) as composite_dst:
            
            for row in range(height):    
                chunk = stack_src.read(window=Window(0, row, width, 1), masked=True)
                
                # TODO: implement other methods like min/max/mode
                
                centre = np.rint(ma.median(chunk, axis=0)).astype(np.uint16)
                centre_data = centre.data
                centre_data[centre.mask] = NODATA_UINT16   
                composite_dst.write(centre_data, window=Window(0, row, width, 1), indexes=1)

    return composite_path

In [3]:


def create_band_stack(band_name, tif_paths, dst_dir):
    
    with rasterio.open(tif_paths[0]) as meta_src:
        meta = meta_src.meta.copy()

    meta.update(count=len(tif_paths))

    stack_path = f'{dst_dir}/{band_name}_stack.tif'
    with rasterio.open(stack_path, 'w', **meta) as stack_dst:
        for id, layer in enumerate(tif_paths, start=1):
            with rasterio.open(layer) as lyr_src:
                stack_dst.write_band(id, lyr_src.read(1))

    return stack_path


In [5]:
import shutil


def merge_tif_with_blank(tif_path, blank_path, band_name, bbox_poly_ll, merged_path=None):

    if merged_path is None:
        merged_path = tif_path.replace(".tif", "_merged.tif")
    
    with rasterio.open(blank_path) as blank_src:
        with rasterio.open(tif_path) as tif_src:
            
            # if the bounds are the same then just skip merging
            tbnds, dbnds = blank_src.bounds, tif_src.bounds
            if tbnds.left == dbnds.left and tbnds.bottom == dbnds.bottom and \
            tbnds.right == dbnds.right and tbnds.top == dbnds.top:
                shutil.copy2(tif_path, merged_path)
                return merged_path

            merged, transform_ = rasterio.merge.merge([tif_src, blank_src], bounds=bbox_poly_ll.bounds)
            merged = merged[0, :, :]

            merged_profile = blank_src.profile.copy()
            if band_name == "SCL":
                merged_profile["dtype"] = "uint8"
                merged_profile["nodata"] = NODATA_BYTE

    with rasterio.open(merged_path, "w", **merged_profile) as new_src:
        new_src.write(merged, 1)

    return merged_path

In [ ]:


def save_tif_to_s3(task_uid, tif_path, subdir):

    file_name = tif_path.split('/')[-1]
    object_key = f'{task_uid}/{subdir}/{file_name}'
    
    print(f'uploading {tif_path} to s3://{S3_TASKS_BUCKET}/{object_key}')

    s3_utils.put_item(tif_path, S3_TASKS_BUCKET, object_key)

    
    
def save_photo_to_s3(task_uid, photo_path, subdir):

    file_name = photo_path.split('/')[-1]
    object_key = f'{task_uid}/{subdir}/{file_name}'

    print(f'uploading {photo_path} to s3://{S3_TASKS_BUCKET}/{object_key}')

    s3_utils.put_item(photo_path, S3_TASKS_BUCKET, object_key)
    
    
    
def save_tiles_dir_to_s3(task_uid, tiles_dir, subdir):

    print(f'saving {tiles_dir} to S3')

    for root, dirs, files in os.walk(tiles_dir):
        for file in files:
            file_path = os.path.join(root, file)
            subpath = file_path.replace(tiles_dir, '')
            object_key = f'{task_uid}/{subdir}/tiles/{subpath}'
            
            s3_utils.put_item(file_path, S3_TASKS_BUCKET, object_key)


In [ ]:
from osgeo import gdal, gdalconst



def create_vrt(band_paths, dst_path):

    vrt_options = gdal.BuildVRTOptions(separate=True)
    gdal.BuildVRT(dst_path, band_paths, options=vrt_options)


def create_tif(vrt_path, dst_path, isCog=False):

    _format = "COG" if isCog else "GTiff"

    translate_options = gdal.TranslateOptions(
        format=_format, 
        noData=NODATA_UINT16,
    )

    gdal.Translate(dst_path, vrt_path, options=translate_options)


def create_byte_vrt(full_vrt_path, dst_path):

    translate_options = gdal.TranslateOptions(
        format="VRT", 
        outputType=gdalconst.GDT_Byte, 
        scaleParams=[[0, 2000, 0, 255]],
        noData=NODATA_BYTE,
    )
    gdal.Translate(dst_path, full_vrt_path, options=translate_options)
 


In [ ]:


def create_rgb_cog(composites, dst_path):

    vrt_path = '/tmp/temp_rgb.vrt' # TODO: change this... no hard coding paths
    band_paths = [composites['B04'], composites['B03'], composites['B02']]
    create_vrt(band_paths, vrt_path)
    create_tif(vrt_path, dst_path, isCog=True)
    

def create_full_tif(composites, dst_path):

    vrt_path = '/tmp/temp_full.vrt'
    band_paths = [composites[band] for band in composites]
    create_vrt(band_paths, vrt_path)
    create_tif(vrt_path, dst_path, isCog=False)


def create_byte_rgb_vrt(composites, dst_path):

    band_paths = [composites['B04'], composites['B03'], composites['B02']]
    vrt_path = f'/tmp/temp_rgb_byte_.vrt'
    create_vrt(band_paths, vrt_path)
    create_byte_vrt(vrt_path, dst_path)


In [ ]:


def create_rgb_map_tiles(rgb_vrt, step):
    tiles_dir = f'/tmp/{step}/tiles/' # TODO: no hard coding paths
    print(f'generating tiles from {rgb_vrt}: {tiles_dir}')

    options = {
        'kml': True,
        'nb_processes': 4,
        'title': 'Smart Carte',
        'zoom': (2, 12),
    }

    gdal2tiles.generate_tiles(rgb_vrt, tiles_dir, **options)
    return tiles_dir
